In [72]:
import tensorflow as tf
from tensorflow.contrib import rnn

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data', one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [78]:
tf.reset_default_graph()

learning_rate = 0.01
training_steps = 10000
batch_size = 128
display_step = 200

num_inputs = 28
timesteps = 28
num_units = 128
num_classes = 10


X = tf.placeholder(tf.float32, [None, timesteps, num_inputs])
Y = tf.placeholder(tf.float32, [None, num_classes])

In [79]:
weights = {
    'out': tf.Variable(tf.random_normal([num_units, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [80]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_units, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [81]:

logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
train_op = optimizer.minimize(loss_op)

correct_pred = tf.equal(tf.argmax(prediction,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

In [82]:
with tf.Session() as sess:
    sess.run(init)
    
    for steps in range(1, training_steps+1):
        x_batch, y_batch = mnist.train.next_batch(batch_size)
        x_batch = x_batch.reshape((batch_size, timesteps, num_inputs))
        sess.run([train_op], feed_dict = {X: x_batch, Y: y_batch})
        
        if steps %200 == 0:
            loss, acc = sess.run([loss_op, accuracy], feed_dict = {X: x_batch, Y: y_batch})
            print("iter", steps, "minibatch loss", loss, "minibatch accuracy", acc)
    
    print("training finished!")
    
    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_inputs))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

iter 200 minibatch loss 1.1572338 minibatch accuracy 0.6328125
iter 400 minibatch loss 0.8960469 minibatch accuracy 0.75
iter 600 minibatch loss 0.64494306 minibatch accuracy 0.8125
iter 800 minibatch loss 0.4541263 minibatch accuracy 0.890625
iter 1000 minibatch loss 0.453861 minibatch accuracy 0.859375
iter 1200 minibatch loss 0.35250735 minibatch accuracy 0.8984375
iter 1400 minibatch loss 0.37073123 minibatch accuracy 0.8671875
iter 1600 minibatch loss 0.32649475 minibatch accuracy 0.875
iter 1800 minibatch loss 0.23312849 minibatch accuracy 0.9375
iter 2000 minibatch loss 0.20754817 minibatch accuracy 0.9296875
iter 2200 minibatch loss 0.14087595 minibatch accuracy 0.9609375
iter 2400 minibatch loss 0.13877735 minibatch accuracy 0.9609375
iter 2600 minibatch loss 0.1724372 minibatch accuracy 0.9453125
iter 2800 minibatch loss 0.19115505 minibatch accuracy 0.921875
iter 3000 minibatch loss 0.15169433 minibatch accuracy 0.9609375
iter 3200 minibatch loss 0.14120626 minibatch accurac